# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 15 2022 10:27AM,244146,22,624238,"NBTY Global, Inc.",Released
1,Aug 15 2022 10:27AM,244145,22,624235,"NBTY Global, Inc.",Released
2,Aug 15 2022 10:25AM,244143,15,8404182,"Carwin Pharmaceutical Associates, LLC",Released
3,Aug 15 2022 10:25AM,244143,15,8404804,"Carwin Pharmaceutical Associates, LLC",Released
4,Aug 15 2022 10:24AM,244142,15,8395082,"Mizner Bioscience, LLC",Released
5,Aug 15 2022 10:22AM,244140,15,30009936,"Alliance Pharma, Inc.",Released
6,Aug 15 2022 10:22AM,244140,15,30009937,"Alliance Pharma, Inc.",Released
7,Aug 15 2022 10:22AM,244140,15,30009938,"Alliance Pharma, Inc.",Released
8,Aug 15 2022 10:22AM,244140,15,30009939,"Alliance Pharma, Inc.",Released
9,Aug 15 2022 10:22AM,244140,15,30009940,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
17,244141,Released,21
18,244142,Released,1
19,244143,Released,2
20,244145,Released,1
21,244146,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
244141,NaN,21.0
244142,NaN,1.0
244143,NaN,2.0
244145,NaN,1.0
244146,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
244108,6.0,5.0
244109,0.0,27.0
244111,0.0,1.0
244121,0.0,3.0
244122,0.0,6.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
244108,6,5
244109,0,27
244111,0,1
244121,0,3
244122,0,6


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,244108,6,5
1,244109,0,27
2,244111,0,1
3,244121,0,3
4,244122,0,6


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,244108,6,5
1,244109,,27
2,244111,,1
3,244121,,3
4,244122,,6


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 15 2022 10:27AM,244146,22,"NBTY Global, Inc."
1,Aug 15 2022 10:27AM,244145,22,"NBTY Global, Inc."
2,Aug 15 2022 10:25AM,244143,15,"Carwin Pharmaceutical Associates, LLC"
4,Aug 15 2022 10:24AM,244142,15,"Mizner Bioscience, LLC"
5,Aug 15 2022 10:22AM,244140,15,"Alliance Pharma, Inc."
18,Aug 15 2022 10:19AM,244141,15,"Brookfield Pharmaceuticals, LLC"
39,Aug 15 2022 10:09AM,244139,10,Bio-PRF
47,Aug 15 2022 10:08AM,244138,10,Eywa Pharma Inc.
53,Aug 15 2022 9:43AM,244137,16,"SHL Pharma, LLC"
54,Aug 15 2022 9:28AM,244135,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Aug 15 2022 10:27AM,244146,22,"NBTY Global, Inc.",,1
1,Aug 15 2022 10:27AM,244145,22,"NBTY Global, Inc.",,1
2,Aug 15 2022 10:25AM,244143,15,"Carwin Pharmaceutical Associates, LLC",,2
3,Aug 15 2022 10:24AM,244142,15,"Mizner Bioscience, LLC",,1
4,Aug 15 2022 10:22AM,244140,15,"Alliance Pharma, Inc.",,13
5,Aug 15 2022 10:19AM,244141,15,"Brookfield Pharmaceuticals, LLC",,21
6,Aug 15 2022 10:09AM,244139,10,Bio-PRF,,8
7,Aug 15 2022 10:08AM,244138,10,Eywa Pharma Inc.,,6
8,Aug 15 2022 9:43AM,244137,16,"SHL Pharma, LLC",,1
9,Aug 15 2022 9:28AM,244135,10,ISDIN Corporation,,10


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 15 2022 10:27AM,244146,22,"NBTY Global, Inc.",1,
1,Aug 15 2022 10:27AM,244145,22,"NBTY Global, Inc.",1,
2,Aug 15 2022 10:25AM,244143,15,"Carwin Pharmaceutical Associates, LLC",2,
3,Aug 15 2022 10:24AM,244142,15,"Mizner Bioscience, LLC",1,
4,Aug 15 2022 10:22AM,244140,15,"Alliance Pharma, Inc.",13,
5,Aug 15 2022 10:19AM,244141,15,"Brookfield Pharmaceuticals, LLC",21,
6,Aug 15 2022 10:09AM,244139,10,Bio-PRF,8,
7,Aug 15 2022 10:08AM,244138,10,Eywa Pharma Inc.,6,
8,Aug 15 2022 9:43AM,244137,16,"SHL Pharma, LLC",1,
9,Aug 15 2022 9:28AM,244135,10,ISDIN Corporation,10,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 15 2022 10:27AM,244146,22,"NBTY Global, Inc.",1,
1,Aug 15 2022 10:27AM,244145,22,"NBTY Global, Inc.",1,
2,Aug 15 2022 10:25AM,244143,15,"Carwin Pharmaceutical Associates, LLC",2,
3,Aug 15 2022 10:24AM,244142,15,"Mizner Bioscience, LLC",1,
4,Aug 15 2022 10:22AM,244140,15,"Alliance Pharma, Inc.",13,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 15 2022 10:27AM,244146,22,"NBTY Global, Inc.",1.0,NaN
1,Aug 15 2022 10:27AM,244145,22,"NBTY Global, Inc.",1.0,NaN
2,Aug 15 2022 10:25AM,244143,15,"Carwin Pharmaceutical Associates, LLC",2.0,NaN
3,Aug 15 2022 10:24AM,244142,15,"Mizner Bioscience, LLC",1.0,NaN
4,Aug 15 2022 10:22AM,244140,15,"Alliance Pharma, Inc.",13.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 15 2022 10:27AM,244146,22,"NBTY Global, Inc.",1.0,0.0
1,Aug 15 2022 10:27AM,244145,22,"NBTY Global, Inc.",1.0,0.0
2,Aug 15 2022 10:25AM,244143,15,"Carwin Pharmaceutical Associates, LLC",2.0,0.0
3,Aug 15 2022 10:24AM,244142,15,"Mizner Bioscience, LLC",1.0,0.0
4,Aug 15 2022 10:22AM,244140,15,"Alliance Pharma, Inc.",13.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2929504,207.0,6.0
15,976566,37.0,0.0
16,488267,1.0,1.0
20,244122,6.0,0.0
22,488291,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2929504,207.0,6.0
1,15,976566,37.0,0.0
2,16,488267,1.0,1.0
3,20,244122,6.0,0.0
4,22,488291,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,207.0,6.0
1,15,37.0,0.0
2,16,1.0,1.0
3,20,6.0,0.0
4,22,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,207.0
1,15,Released,37.0
2,16,Released,1.0
3,20,Released,6.0
4,22,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,20,22
Status,,,,,
Executing,6.0,0.0,1.0,0.0,0.0
Released,207.0,37.0,1.0,6.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,20,22
0,Executing,6.0,0.0,1.0,0.0,0.0
1,Released,207.0,37.0,1.0,6.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,20,22
0,Executing,6.0,0.0,1.0,0.0,0.0
1,Released,207.0,37.0,1.0,6.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()